# Face Expression Detection with YOLOv8

This notebook contains the complete workflow for training, evaluating, and deploying a facial expression detection model using YOLOv8 on the AffectNet dataset.

In [ ]:
# Install dependencies
!pip install ultralytics streamlit pillow -q

In [ ]:
# Import libraries
import os
import shutil
from ultralytics import YOLO
import streamlit as st
from PIL import Image

In [ ]:
# 1. Prepare Dataset
INPUT_DATA_PATH = "/kaggle/input/affectnet-yolo-format/YOLO_format"  # Adjust for local path if needed
WORKING_DATA_PATH = "/kaggle/working/YOLO_format"

if os.path.exists(WORKING_DATA_PATH):
    shutil.rmtree(WORKING_DATA_PATH)
shutil.copytree(INPUT_DATA_PATH, WORKING_DATA_PATH)

# Create and update affectnet.yaml
yaml_content = """
path: {}
train: train/images
val: valid/images
test: test/images
nc: 8
names: ['neutral', 'happy', 'sad', 'surprise', 'fear', 'disgust', 'anger', 'contempt']
""".format(WORKING_DATA_PATH)

with open("affectnet.yaml", "w") as f:
    f.write(yaml_content)

# Fix label files
label_dir = os.path.join(WORKING_DATA_PATH, "train/labels")
for label_file in os.listdir(label_dir):
    label_path = os.path.join(label_dir, label_file)
    with open(label_path, 'r') as f:
        lines = f.readlines()
    with open(label_path, 'w') as f:
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 0 and int(parts[0]) == 7:
                f.write(line)
            elif len(parts) > 0 and int(parts[0]) < 7:
                f.write(line)

In [ ]:
# 2. Train the Model
model = YOLO("yolov8n.pt")
results = model.train(
    data="affectnet.yaml",
    epochs=50,
    imgsz=224,
    batch=32,
    project="models/runs/detect",
    name="train",
    exist_ok=True
)
print("Training completed.")

In [ ]:
# 3. Evaluate the Model
model = YOLO("runs/detect/train/weights/best.pt")
results = model.val(data="affectnet.yaml")
print("Metrik Evaluasi:")
print(f"mAP@50: {results.box.map}")
print(f"Precision: {results.box.map}")
print(f"Recall: {results.box.map}")

In [ ]:
# 4. Export Model to ONNX
model.export(format="onnx")
print("Model exported to ONNX format.")

### Deployment using streamlit (optional) Or just using deploy.py

```python

In [ ]:
# 5. Save Streamlit Code for Local Deployment
streamlit_code = '''import streamlit as st
from ultralytics import YOLO
from PIL import Image
import os

model_path = "models/runs/detect/train/weights/best.pt"
if not os.path.exists(model_path):
    st.error("Model file not found. Please upload 'best.pt' to the models directory.")
else:
    model = YOLO(model_path)
    st.title("Deteksi Ekspresi Wajah")
    uploaded_file = st.file_uploader("Unggah Gambar", type=["jpg", "png"])
    if uploaded_file:
        img = Image.open(uploaded_file)
        results = model(img)
        st.image(results.render()[0], caption="Hasil Deteksi")
'''

with open("deploy.py", "w") as f:
    f.write(streamlit_code)
print("Streamlit code saved to deploy.py. Run 'streamlit run deploy.py' locally.")

## Notes
- Run each cell sequentially to execute the full pipeline.
- For local execution, adjust `INPUT_DATA_PATH` to the local dataset path.
- Download `best.pt` and `deploy.py` from the output directory for deployment.